# Import Modules

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import ptitprince as pt
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, PandasTools, MolFromSmiles
from tqdm import tqdm
tqdm.pandas()

# Load data file

In [2]:
surechem_df = pd.read_csv('data/EDA_df.txt.gz', sep='\t', compression='gzip')

In [3]:
surechem_df['year'] = surechem_df['PUBLICATION_DATE'].progress_apply(lambda x: x.split('-')[0])

100%|██████████| 133512452/133512452 [02:21<00:00, 946075.70it/s] 


In [4]:
surechem_df = surechem_df.drop_duplicates(subset=["InChIKey","year"], keep='first')
surechem_df.reset_index(drop=True, inplace=True)
len(surechem_df)

21857225

In [5]:
year_df = surechem_df[['SMILES', 'year']]

In [6]:
with open('data/properties.json', 'r') as f:
    properties_dict = json.load(f)

In [7]:
properties_df = pd.DataFrame(properties_dict)
properties_df = properties_df.T
properties_df.reset_index(inplace=True)
properties_df.columns = ['SMILES', 'mw', 'logp', 'n_hba', 'n_hbd', 'rot_bonds', 'tpsa', 'fsp3', 'n_chiral']
properties_df.head(2)

,SMILES,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral
0,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,217.077264,0.6279,3.0,2.0,3.0,57.61,0.777778,1.0
1,O=C(O)\C=C/C(=O)O.CCOC(=O)[C@H](CCC1=CC=CC=C1)...,492.210781,1.3164,7.0,4.0,11.0,170.54,0.458333,1.0


In [8]:
len(properties_df)

1505470

In [9]:
merged_df = pd.merge(properties_df, year_df, how="left", on=["SMILES"])
merged_df.head(2)

,SMILES,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral,year
0,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,217.077264,0.6279,3.0,2.0,3.0,57.61,0.777778,1.0,2015
1,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,217.077264,0.6279,3.0,2.0,3.0,57.61,0.777778,1.0,2016


In [10]:
len(merged_df)

4695795

In [11]:
liste = []
for idx, row in merged_df.iterrows():
    if '.' not in row[0]:
        df = merged_df.iloc[idx]
        liste.append(df) 

In [12]:
smiles_without_dots = pd.DataFrame(liste)

In [13]:
len(smiles_without_dots)

4329398

In [14]:
smiles_with_dots = len(merged_df)- len(smiles_without_dots) 

In [15]:
smiles_with_dots #salt compounds gave different properties, hence, removed

366397

# formatting calculated values

In [16]:
pd.options.display.float_format = '{:.2f}'.format

# concatenating dataframes

In [17]:
def define_year(year):
    df_year = smiles_without_dots[(smiles_without_dots['year'] == str(year))]
    return df_year

In [18]:
def year_range(year1, year2):
    concat_df = None
    for year in range(year1, year2):
        current_df = define_year(year)

        if year == 2015:
            df = smiles_without_dots[(smiles_without_dots['year'] == str(year))]
            concat_df = current_df
        else:
            concat_df = pd.concat([concat_df, current_df])
            concat_df.drop_duplicates(keep='first', inplace=True)
            concat_df.reset_index(drop=True, inplace=True)
    return concat_df

# Year 2015-2017

In [19]:
concat_df_15_16_17 = year_range(2015, 2018)

In [20]:
concat_df_15_16_17 = concat_df_15_16_17[['mw', 'logp', 'n_hba', 'n_hbd', 'rot_bonds','tpsa','fsp3','n_chiral']]

In [28]:
concat_df_15_16_17.astype(float).describe()

,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral
count,2586036.00,2586036.00,2586036.00,2586036.00,2586036.00,2586036.00,2586036.00,2586036.00
mean,365.39,3.58,4.50,1.40,5.50,71.18,0.40,3.41
std,177.69,2.89,2.86,1.57,5.41,49.38,0.27,36.40
min,100.01,-27.34,0.00,0.00,0.00,0.00,0.00,1.00
25%,230.97,1.95,2.00,0.00,2.00,38.69,0.20,1.00
50%,345.14,3.23,4.00,1.00,4.00,64.79,0.36,1.00
75%,459.16,4.64,6.00,2.00,7.00,93.95,0.56,2.00
max,4863.19,114.29,108.00,61.00,256.00,1788.74,1.00,1024.00


# Year 2018-2020

In [22]:
concat_df_18_19_20 = year_range(2018, 2021)

In [23]:
concat_df_18_19_20 = concat_df_18_19_20[['mw', 'logp', 'n_hba', 'n_hbd', 'rot_bonds','tpsa','fsp3','n_chiral']]

In [24]:
concat_df_18_19_20.astype(float).describe()

,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral
count,1191215.00,1191215.00,1191215.00,1191215.00,1191215.00,1191215.00,1191215.00,1191215.00
mean,329.80,3.29,4.04,1.36,5.23,65.76,0.42,3.04
std,171.37,2.91,2.76,1.58,5.62,48.62,0.29,31.67
min,100.01,-27.34,0.00,0.00,0.00,0.00,0.00,1.00
25%,200.06,1.64,2.00,0.00,2.00,34.14,0.19,1.00
50%,291.03,2.90,3.00,1.00,4.00,57.61,0.37,1.00
75%,424.04,4.38,5.00,2.00,7.00,88.29,0.62,2.00
max,4292.75,114.29,77.00,61.00,256.00,1757.47,1.00,1024.00


# Year 2021-2022

In [25]:
concat_df_21_22 = year_range(2021, 2023)

In [26]:
concat_df_21_22 = concat_df_21_22[['mw', 'logp', 'n_hba', 'n_hbd', 'rot_bonds','tpsa','fsp3','n_chiral']]

In [27]:
concat_df_21_22.astype(float).describe()

,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral
count,536041.00,536041.00,536041.00,536041.00,536041.00,536041.00,536041.00,536041.00
mean,305.26,3.09,3.72,1.29,5.03,61.37,0.44,3.06
std,163.88,2.94,2.68,1.55,5.73,47.50,0.31,32.42
min,100.01,-24.36,0.00,0.00,0.00,0.00,0.00,1.00
25%,186.05,1.46,2.00,0.00,2.00,29.26,0.18,1.00
50%,258.00,2.64,3.00,1.00,4.00,52.37,0.39,1.00
75%,390.06,4.13,5.00,2.00,6.00,83.10,0.67,2.00
max,4292.75,114.29,77.00,58.00,256.00,1690.64,1.00,1024.00
